In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.feature_extraction.text import CountVectorizer

# Dataset uploading

In [2]:
import os 

DATASET_PATHS = {
    "local": {
        "train": "../../datasets/train_set.csv",
        "test": "../../datasets/test_set.csv"
    },
    "kaggle": {
        "train": "/kaggle/input/python-codes-time-complexity/train_set.csv",
        "test": "/kaggle/input/python-codes-time-complexity/test_set.csv"
    }
}

def upload_datasets(dataset_paths=DATASET_PATHS):
    for path in dataset_paths:
        if os.path.exists(dataset_paths[path]['train']) and os.path.exists(dataset_paths[path]['test']):
            return dataset_paths[path]['train'], dataset_paths[path]['test']

    return FileNotFoundError(f"Datasets do not exist in the current paths: {dataset_paths}")
            

train_set_path, test_set_path = upload_datasets()

# Dataset Uploading

In [3]:
train_set = pd.read_csv(train_set_path)
test_set = pd.read_csv(test_set_path)

X_train = train_set['code']
y_train = train_set['complexity']

X_test = test_set['code']
y_test = test_set['complexity']

# Metrics

## Hierarchy Complexity Score

### Ordering labels by Hierarchy

In [4]:
LABELS_HIERARCHY = {
    'constant': 1,
    'logn': 2,
    'linear': 3,
    'nlogn': 4,
    'quadratic': 5,
    'cubic': 6,
    'np': 7
}

N_CLASSES = len(LABELS_HIERARCHY)

### Writing the custom metric hc_score

In [5]:
def hc_score(y_true, y_pred, n_classes=N_CLASSES):
    assert len(y_true) == len(y_pred), f"The amount of y_true labels: {len(y_true)} does not equal to the amount of y_pred: {len(y_pred)}."

    y_pred = np.array([LABELS_HIERARCHY[pred] for pred in y_pred])
    y_true = np.array([LABELS_HIERARCHY[true] for true in y_true])
    n_samples = len(y_true)
    
    return (np.sum(np.abs(y_pred - y_true)) / n_classes) / n_samples

hierarchy_score = make_scorer(hc_score)

### All metrics

In [6]:
metrics = {'accuracy': 'accuracy', 'f1_macro': 'f1_macro', 'hc_score': hierarchy_score}

# Model & Cross-Validation

In [7]:
model = LogisticRegression(max_iter=1000, random_state=2025)
pipeline = Pipeline([('tokenizer', CountVectorizer()), ('model', model)])

scores = cross_validate(pipeline, X_train, y_train, scoring=metrics, cv=5)

In [8]:
cv_accuracy = scores['test_accuracy'].mean()
cv_f1_macro = scores['test_f1_macro'].mean()
cv_hc_score = scores['test_hc_score'].mean()

print(f"cv_accuracy: {cv_accuracy}")
print(f"cv_f1_macro: {cv_f1_macro}")
print(f"cv_hc_score: {cv_hc_score}")

cv_accuracy: 0.6295055086835667
cv_f1_macro: 0.6326589275313579
cv_hc_score: 0.1043944880967546


# Evaluation

In [15]:
pipeline.fit(X_train, y_train)
y_preds = pipeline.predict(X_test)

test_accuracy = round(accuracy_score(y_test, y_preds) * 100, 2)
test_f1_macro = round(f1_score(y_test, y_preds, average='macro') * 100, 2)
test_hc_score = round(hc_score(y_test, y_preds) * 100, 2)

print(f"test_accuracy: {test_accuracy}")
print(f"test_f1_macro: {test_f1_macro}")
print(f"test_hc_score: {test_hc_score}")

test_accuracy: 66.77
test_f1_macro: 67.31
test_hc_score: 8.94


# Save performance

In [16]:
results = pd.DataFrame(data={'model': 'LogisticRegression', 'accuracy': test_accuracy,
                             'f1_macro': test_f1_macro, 'hc_score (tltb)': test_hc_score}, index=[0])
results.to_csv("models_leaderboard.csv", index=False)